# Finding Hidden Message in DNA
## Week 3

**"Clock" Gene**

Circadian clock controls daily schedules of animals, plants, and even bacteria. Today many genes have been discovered that play a role in regulating circadian rhythm.

First we will focus on plants, as circadian clock is a matter of life or death. Estimates on plant circadian genes are over a thousand, related to photosynthesis, photo reception, and flowering. Every plant cell keeps track of day/night independently of other cells. Genes LHY, CCA1, and TOC1 are the master time keeping regulatory genes and the regulatory proteins they encode are often controlled by external factors (nutrients, sunlight, etc). These regulatory proteins are **transcription factors** that inhibit or promotes expression of genes. A transcription factor bind to a specific short DNA interval called a **regulatory motif or transcription factor binding site** in the genes upstream region (600-1000bp).

Since these regulatory motifs are not completely conserved how can we locate them without knowing what they look like in advance?
* We could utilize the Frequent word with mismatches algorithm used in week 2, however this was aimed at shorter k-mers with fewer mismatches. Plus finding a DnaA box which is a pattern that appears frequently in a relatively short interval of the genome, whereas a regulatory motif is a pattern that appears at least once (with possible variation) in many different regions scattered throughout the genome.

In [1]:
# Helper Functions from previous weeks
def symToNum(symbol):
    symMap = {'A': 0,
              'C': 1,
              'G': 2,
              'T': 3}
    return symMap[symbol]

def numToSym(num):
    numMap = {0: 'A',
              1: 'C',
              2: 'G',
              3: 'T'}
    return numMap[num]

def numToPat(index, k):
    if k == 1:
        return numToSym(index)
    pfixIndex = int(index / 4)
    r = index % 4
    sym = numToSym(r)
    pfixPattern = numToPat(pfixIndex, k - 1)
    return pfixPattern + sym

def patToNum(pattern):
    if not pattern:
        return 0
    symbol = pattern[-1]
    prefix = pattern[:-1]
    return 4 * patToNum(prefix) + symToNum(symbol)

def hammDist(seq1, seq2, d=-1):
    dist = 0
    if len(seq1) != len(seq2):
        return -1
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            dist += 1
        if d != -1 and dist > d:
            return -1
    return dist

def neighbors(pattern, d):
    nucleos = {'A', 'C', 'G', 'T'}
    if d == 0:
        return pattern
    if len(pattern) == 1:
        return nucleos
    hood = set()
    sufNeighbors = neighbors(pattern[1:], d)
    for neighbor in sufNeighbors:
        if hammDist(pattern[1:], neighbor) < d:
            for nucleotide in nucleos:
                hood.add(nucleotide + neighbor)
        else:
            hood.add(pattern[:1] + neighbor)
    return hood

In [7]:
def motifEnumeration(Dna, k, d):
    '''
    Given a collection of strings, integers k, and d. Find all motifs (kmers) that are k length and
        appear in all strings within the collection with no more than d mismatches. Brute-Force Method.
        
    Args:
        Dna (list[string]): Collection of DNA sequences
        k (int): Length of motif(s) we would like to find
        d (int): Max mismatches (distance) between k-mer and matches.
        
    Returns:
        list[string]: all k-mers (motif(s)) that appear in each sequence within the Dna collection
            allowing no more than d mismatches.
    '''
    patterns = set()
    for i in range(len(Dna[0]) - k + 1):
        kmer = Dna[0][i:i+k]
        hood = neighbors(kmer, d)
        for candidate in hood:
            check = 0
            for seq in Dna:
                check = 0
                for i in range(len(seq) - k + 1):
                    tk = seq[i:i+k]
                    if hammDist(tk, candidate, d) > -1:
                        check = 1
                        break
                if check != 1:
                    break
            if check:
                patterns.add(candidate)
    return list(patterns)

In [37]:
# test/run motifEnumeration function

# test = ['ATTTGGC', 'TGCCTTA', 'CGGTATC', 'GAAAATT']
# print(motifEnumeration(test, 3, 1))

with open("./Data/dataset_156_8.txt") as inFile:
    data = inFile.readlines()

params = data[0].strip().split(' ')
dna = []
for i in range(1, len(data)):
    dna.append(data[i].strip())

print(*motifEnumeration(dna, int(params[0]), int(params[1])))

GGCCG GGCTG AGGCG CGCGG CGGCG GGAGG TGGCG GGCAG GGCGG GCCGG GGCGC GGGCG


Think about the run time of the brute force algorithm:
1. Runs through each kmer of the first DNA seq in the collection (len(seq) - k + 1)
2. Calculates all neighbors for the kmer
3. For each neighbor, examine all kmers of each subsequent DNA seq in the collection and see if there is a match within d distance.

$O(n^t \times k \times t)$ Needless to say this algorithm would not scale well at all for larger values of k or d

What can we do differently?

**Scoring Motifs**

* Consider a list of t DNA strings(Dna), where each string has length n, and select a k-mer from each string to form a collection Motifs.


1. Score(Motifs) => # of unpopular (non-consensus) letters in a column of the motif matrix
2. Count(Motifs) => Construct the 4 x k count matrix. Count the number of occurrences of each nucleotide in each column of the motif matrix
3. Profile(Motifs) => Divide each element of the count matrix by t
4. Generate a consensus string from the most popular letter in each column of the motif matrix

<img src="img/motifs_score_count_profile_consensus.png" width="500">

How is conservation accounted for? Think about column 2 (6C, 2A, 2T) and column 12 (6C, and 4T). They both contribute 4 to Score, does this make sense biologically?
* No, nucleotides with frequencies exceeding 0.4 should be weighted differently. In the example column 12 is "more conserved"

Every column of the profile matrix corresponds to a probability distribution (collection of non-negative numbers that sum to 1).

**Entropy** is a measure of uncertainty of a probability distribution

**$H = -\sum \limits_{i=1}^{N} p_{i} * log_{2}p_{i}$**

Example:
* $log_{2}(0)$ is undefined but multiplied by 0 we assume is 0
* Row 2 = $-(0.2log_{2}(0.2) + 0.6log_{2}(0.6) + 0.0log_{2}(0.0) + 0.2log_{2}(0.2)) =1.371$
* Row 12 = $-(0.0log_{2}(0.0) + 0.6log_{2}(0.6) + 0.0log_{2}(0.0) + 0.4log_{2}(0.4)) =0.971$

Minimum Entropy = 0, where all nucleotides are completely conserved
Maximum Entropy = 2, where all probabilities are 1/4

The more conserved = the smaller the entropy

Instead of exploring all Motifs in Dna and deriving the consensus string from Motifs afterwards. We can explore all potential k-mer consensus strings first and then find the best possible collection Motifs for each consensus string.

**Median String Problem**

Double minimization problem, we must find a k-mer Pattern that minimizes d(Pattern, Dna) where this function itself is computed by taking a minimum over all choices of k-mers from each string in Dna.


In [10]:
def distPatString(Pattern, Dna):
    '''
    Distance Between Pattern and String.
    Finds the minimum distance between pattern and each string within the Dna collection.
    
    Args:
        Pattern (string): Pattern to search for.
        Dna (list[string]): Collection of strings to search for pattern within.
        
    Returns:
        int: Minimum distance between pattern and all strings within the collection.
    '''
    k = len(Pattern)
    dist = 0
    for string in Dna:
        hamDist = float('inf')
        for i in range(len(string) - k + 1):
            tmp = hammDist(Pattern, string[i:i+k])
            if hamDist > tmp:
                hamDist = tmp
        dist += hamDist
    return dist

In [11]:
# test/run distPatString function

pat = 'AAA'
Dna = ['TTACCTTAAC', 'GATATCTGTC', 'ACGGCGTTCG', 'CCCTAAAGAG', 'CGTCAGAGGT']
print(distPatString(pat, Dna))

# with open("./Data/dataset_5164_1.txt") as inFile:
#     data = inFile.readlines()

# Dna = data[1].strip().split(" ")
# print(distPatString(data[0].strip(), Dna))

5


In [16]:
def medString(Dna, k, candidates=False):
    '''
    Median String Problem.
    Find a motif of length k, that has the smallest distance between all of the strings in collection Dna.
    
    Args:
        Dna (list[string]): Collection of DNA sequnces.
        k (int): Length of motif to find.
        candidates (bool, optional): Return all candidates.
            Defualts to false, and returns only the first smallest string.
            If true and there are multiple minimum strings returns list of all candidate strings.
            
    Returns:
        string: Sequence that is of length k and has the smallest distance within all sequneces of the collection.
    '''
    dist = float('inf')
    cans = []
    for i in range((4 ** k) - 1):
        pat = numToPat(i, k)
        cDist = distPatString(pat, Dna)
        if dist > cDist:
            dist = cDist
            med = pat
            cans = [med]
        elif dist == cDist:
            cans.append(pat)
    if candidates and len(cans) > 1:
        return cans
    else:
        return med

In [17]:
# test/run medString function

k = 3
Dna = ['AAATTGACGCAT', 'GACGACCACGTT', 'CGTCAGCGCCTG', 'GCTGAGCACCGG', 'AGTTCGGGACAG']
print(medString(Dna, k))

# with open("./Data/dataset_158_9.txt") as inFile:
#     data = inFile.readlines()

# k = int(data[0].strip())
# dna = []
# for i in range(1, len(data)):
#     dna.append(data[i].strip())
# print(medString(dna,k))

GAC


The median string function works well enough, yet still has a runtime of $O(4^k \times n \times k \times t)$ for larger values of k this solution may be too slow.

**Greedy Algorithms**

Chooses most attractive alternative at each iteration, optimization occurs over each iteration not globally

In [21]:
def profileMost(Text, k, Profile):
    '''
    Profile-Most Probable k-mer Problem.
    Finds k-mer most likely to be generated from the given profile within Text.
    
    Args:
        Text (string): Text to search through.
        k (int): Length of target motif.
        Profile (list[float]): Nucleotide probability profile for each position in k.
        
    Returns:
        string: Profile most probable k-mer within Text.
    '''
    profMax = 0.0
    kmerMax = Text[:k]
    for i in range(1, len(Text) - k + 1):
        kmer = Text[i:i+k]
        tProf = 1.0
        for x in range(k):
            tProf *= Profile[symToNum(kmer[x])][x]
        if tProf > profMax:
            profMax = tProf
            kmerMax = kmer
    return kmerMax

In [22]:
# test/run profile function

prof = [[0.2, 0.2, 0.3, 0.2, 0.3], [0.4, 0.3, 0.1, 0.5, 0.1], [0.3, 0.3, 0.5, 0.2, 0.4], [0.1, 0.2, 0.1, 0.1, 0.2]]
print(profileMost("ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT", 5, prof))

# with open("./Data/dataset_159_3.txt") as inFile:
#     data = inFile.readlines()

# text = data[0].strip()
# k = int(data[1].strip())
# prof = []
# for i in range(2,len(data)):
#     prof.append([float(x) for x in data[i].strip().split(" ")])

# print(profile(text, k, prof))

CCGAG


In [42]:
# Greedy Motif Search

def makeProfile(Dna, pCounts=False):
    '''
    Generates a nucleotide probability profile for a given collection of same length sequences.
    
    Args:
        Dna (list[string]): Collection of sequences to use for probability calculations
        pCounts (bool, optional): Include pseudocounts.
            Defaults to False, allows zero probabilities.
            If true pseudocounts are introduced according to Laplace's rule of succession.
            
    Returns:
        list[float]: Nucleotide probability profile for given collection of sequences.
    '''
    prof = [[0] * len(Dna[0]) for _ in range(4)]
    for string in Dna:
        for i in range(len(string)):
            prof[symToNum(string[i])][i] += 1
    if pCounts:
        return [[(x+1)/(len(Dna) + 4) for x in i] for i in prof]
    else:
        return [[x/len(Dna) for x in i] for i in prof]

def makeCon(prof):
    '''
    Generate Consensus sequence from a given nucleotide probability profile.
    
    Args:
        prof (list[float]): Nucleotide probability profile.
    
    Returns:
        string: Consensus sequence for the given profile.
    '''
    con = ''
    test = list(zip(*prof))
    for pos in test:
        con += numToSym(pos.index(max(pos)))
    return con

def score(motifs, pCounts=False):
    '''
    Generate a score for a set of motifs.
        Score is the summation of the distance between each motif and the consesnus sequence for a given set.
        
    Args:
        motifs (list[string]): Collection of sequences to score.
        pCounts (bool, optional): Include pseudocounts.
            Defaults to False, allows zero probabilities.
            If true pseudocounts are introduced according to Laplace's rule of succession.
        
    Returns:
        int: Score of the motif collection.
    '''
    prof = makeProfile(motifs, pCounts)
    con = makeCon(prof)
    score = [hammDist(con, x) for x in motifs]
    return sum(score)

def greedyMotif(Dna, k, t, pCounts=False):
    '''
    Greedy Motif Finding Algorithm.
    Find a set of motifs that minimizes profile most probable string within each string of the collection.
    
    Args:
        Dna (list[string]): Collection of DNA sequences.
        k (int): Length of desired motif.
        t (int): Number of sequences.
        pCounts (bool, optional): Include pseudocounts.
            Defaults to False, allows zero probabilities.
            If true pseudocounts are introduced according to Laplace's rule of succession.
    
    Returns:
        list[string]: Collection of motifs that have best score among the collection of sequences.
    '''
    bestMotifs = [x[0:k] for x in Dna]
    bestScore = score(bestMotifs, pCounts)
    for i in range(1, len(Dna[0]) - k + 1):
        tMotif = [Dna[0][i:i+k]]
        for x in range(1,t):
            prof = makeProfile(tMotif, pCounts)
            tMotif.append(profileMost(Dna[x], k, prof))
        tScore = score(tMotif, pCounts)
        if tScore < bestScore:
            bestMotifs = tMotif
            bestScore = tScore
    return bestMotifs

In [43]:
# test/run greedyMotif function

dna = ['GGCGTTCAGGCA', 'AAGAATCAGTCA', 'CAAGGAGTTCGC', 'CACGTCAATCAC', 'CAATAATATTCG']
print(*greedyMotif(dna, 3, 5, True), sep=" ")

# with open("./Data/dataset_159_5.txt") as inFile:
# with open("./Data/dataset_160_9.txt") as inFile:
#     data = inFile.readlines()
    
# args = data[0].strip().split(" ")
# k = int(args[0])
# t = int(args[1])
# dna = []
# for i in range(1, len(data)):
#     dna.append(data[i].strip())
# print(f"K: {k}\nT: {t}\n")
# print(*greedyMotif(dna, k, t, True), sep=" ")

TTC ATC TTC ATC TTC


In [89]:
# Week 3 Quiz
test = ['CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC', 'GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC', 'GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG']
medString(test, 7, candidates=True)

['AATCCTA', 'GAACCAC', 'GTAGGAA', 'TAGTTTC']


'AATCCTA'